In [1]:
!pip install --upgrade pip --quiet
!pip install cvxopt --quiet

In [1]:
#import some useful packages
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import IBMQ, schedule
from qiskit_ibm_provider import IBMProvider
from qiskit.circuit import Gate, Parameter, QuantumCircuit
from qiskit import pulse, schedule
from qiskit.tools.jupyter import *
from qiskit_ibm_provider.job import job_monitor
from qiskit.pulse import Gaussian
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

%config InlineBackend.figure_formats = ['svg']
import time 
import os
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import csv

# IBMQ.save_account('5baae95ab0c4f46ba156eb0909ff9b8b54fffc1c24b7366190b85d6b2293ed9cac1f51cfa4f008890fcdd9f3871a23d904af20a6812bcf9fedb7fd982d968ca1')
# provider = IBMQ.load_account()

provider = IBMProvider()
backend = provider.get_backend('ibm_brisbane')

In [2]:
#get information about backend
backend_defaults = backend.defaults()
backend_properties = backend.properties()
backend_config = backend.configuration()

In [3]:
#hyperparameters
NUM_SHOTS = 2048

qubit = 109
f01 = backend.properties().qubits[qubit][2].value * 1e9
anhar = backend.properties().qubits[qubit][3].value * 1e9
f12 = f01 + anhar

pi = np.pi
cos = np.cos
sin = np.sin
scale_factor = 1e-7

In [4]:
# amp01 = 0.4150600327660328
amp_halfpi_01 = 0.11532683361128736
amp_halfpi_12 = 0.11769405436165835 
amp01 = 2*amp_halfpi_01
amp12 = 2*amp_halfpi_12
dur12 = 96
sig12 = dur12/4
dur01 = 96
sig01 = dur01/4

## Gaussian Scheds

In [5]:
def S01_sched(phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$G^{(01)}$') as S01_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        with pulse.phase_offset(phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur01,
                                      amp=amp_halfpi_01,
                                      sigma=sig01,
                                      name=r'$G^{(01)}$'), drive_chan)

    return S01_pulse

def S12_sched(phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$G^{(12)}$') as S12_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        with pulse.phase_offset(phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur12,
                                      amp=amp_halfpi_12,
                                      sigma=sig12,
                                      name=r'$G^{(12)}$'), drive_chan)

    return S12_pulse

## DRAG Scheds

In [6]:
def S01_sched(phi):
    with pulse.build(backend=backend) as r01_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Drag(duration=120,
                                      amp=0.18093652143360275/2,
                                      sigma=30,
                                      beta=-1.2643134385128565), drive_chan)

    return r01_pulse

def S12_sched(phi):
    with pulse.build(backend=backend) as r12_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Drag(duration=64,
                                      amp=0.31419172290003516/2,
                                      sigma=16,
                                      beta=-0.5694575558130037), drive_chan)

    return r12_pulse

# Generalize phase tracking protocol with many jobs

In [6]:
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(12)}$') as xpi12_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12,
                            sigma=sig12,
                            name=r'$X_{\pi}^{(12)}$'), drive_chan)

    
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(01)}$') as xpi01_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur01,
                            amp=amp01,
                            sigma=sig01,
                            name=r'$X_{\pi}^{(01)}$'), drive_chan)



In [11]:
def protocol(order):
    # Xpi01 = Gate("$X_{\pi}^{01}$", 1, [])
    # Xpi12 = Gate("$X_{\pi}^{12}$", 1, [])

    # zero = QuantumCircuit(qubit + 1, 1)
    # zero.measure(qubit, 0)

    # one = QuantumCircuit(qubit + 1, 1)
    # one.append(Xpi01, [qubit])
    # one.add_calibration(Xpi01, (qubit,), xpi01_sched, [])
    # one.measure(qubit, 0)

    # two = QuantumCircuit(qubit + 1, 1)
    # two.append(Xpi01, [qubit])
    # two.add_calibration(Xpi01, (qubit,), xpi01_sched, [])
    # two.append(Xpi12, [qubit])
    # two.add_calibration(Xpi12, (qubit,), xpi12_sched, [])
    # two.measure(qubit, 0)

    # discr_circ = [zero, one, two]
    
    with pulse.build(backend=backend) as cal_pi12_sched:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        pulse.play(pulse.Drag(duration=64, amp=0.31419172290003516, sigma=16, beta=-0.4530201342281881), drive_chan)
    weight = 1
    cal_pi12_gate = qiskit.circuit.Gate('cal_pi12_sched', weight, [])

    ground_circ = QuantumCircuit(qubit + 1, 1)
    ground_circ.measure(qubit, 0)

    first_excited_state_circ = QuantumCircuit(qubit + 1, 1)
    first_excited_state_circ.x(qubit)
    first_excited_state_circ.measure(qubit, 0)

    second_excited_state_circ = QuantumCircuit(qubit + 1, 1)
    second_excited_state_circ.x(qubit)
    second_excited_state_circ.append(cal_pi12_gate, [qubit])
    second_excited_state_circ.measure(qubit, 0)
    second_excited_state_circ.add_calibration(cal_pi12_gate, [qubit], cal_pi12_sched)

    discr_circ = [ground_circ, first_excited_state_circ, second_excited_state_circ]
    exp_circs = []
    params_arr = []

    for i in range(97):
        phi_arr = np.random.default_rng().uniform(0, np.pi, len(order))
        params_arr.append(phi_arr)        
        qc = QuantumCircuit(qubit + 1,1)
        gate_arr = []
        for j in range(len(order)):
            level = order[j]
            if level == '1':
                name = r"$G^{01}$" + fr"$(\phi_{j+1})$" 
                # name = r"$G^{01}$" + fr"$({np.round(phi_arr[j], 2)})$" 
                gate_arr.append(Gate(name, 1, []))
                qc.append(gate_arr[j], [qubit])
                qc.add_calibration(gate_arr[j], (qubit,), S01_sched(phi_arr[j]), [])
            elif level == '2':
                name = r"$G^{12}$" + fr"$(\phi_{j+1})$" 
                # name = r"$G^{12}$" + fr"$({np.round(phi_arr[j], 2)})$" 
                gate_arr.append(Gate(name, 1, []))
                qc.append(gate_arr[j], [qubit])
                qc.add_calibration(gate_arr[j], (qubit,), S12_sched(phi_arr[j]), [])
            else:
                print('Invalid level')
                break
        qc.measure(qubit,0)
        exp_circs.append(qc)
        
    circs = discr_circ + exp_circs
    
    return circs, params_arr

In [23]:
qubit

109

In [8]:
#create a list of random order
import random
def create_order(length):
    order = ''
    for i in range(length):
        order = order + str(random.randint(1,2))
    return order

def create_orders(length):
    orders = set()
    for i in range(20):
        orders.add(create_order(length))
    return orders


In [9]:
orders = []
for i in range(6, 9, 1):
    orders += random.sample(list(create_orders(i)), k=1)
    
#run many jobs with many orders
tg = time.localtime(time.time())
dd = str(tg.tm_mday)
mm = str(tg.tm_mon) 
yy = str(tg.tm_year)
folder = f"{yy}_{mm}_{dd}"
print(orders)

['121111', '1122121', '22121222']


In [13]:
id_list = []
for order in orders:
    exp_circs, params = protocol(order)
    tg = time.localtime(time.time())
    params_file = f"./{folder}/{order}_params_{yy}_{mm}_{dd}_brisbane.csv"
    np.savetxt(params_file, params, delimiter=',')
    job = backend.run(exp_circs, meas_level=1, meas_return='single', shots=NUM_SHOTS)
    id_list.append(job.job_id())

In [14]:
#save job's id and orders

jobid_file = f"./{folder}/job_id_{yy}_{mm}_{dd}_brisbane.txt"
with open(jobid_file, 'w') as f:
    for job_id in id_list:
        f.write(job_id)
        f.write('\n')

file = f"./{folder}/order_{yy}_{mm}_{dd}_brisbane.txt"
with open(file, 'w') as f:
    for order in orders:
        f.write(order)
        f.write('\n')